# Document Clustering and Topic Modeling 

*In* this project, we use unsupervised learning models to cluster unlabeled documents into different groups, visualize the results and identify their latent topics/structures.

# Part 1: Load Data

In [1]:
import numpy as np
import pandas as pd
import nltk
# import gensim

from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt

nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\86186\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\86186\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
# Load data into dataframe
df = pd.read_csv('watch_reviews.tsv', sep='\t', error_bad_lines=False)

b'Skipping line 8704: expected 15 fields, saw 22\nSkipping line 16933: expected 15 fields, saw 22\nSkipping line 23726: expected 15 fields, saw 22\n'
b'Skipping line 85637: expected 15 fields, saw 22\n'
b'Skipping line 132136: expected 15 fields, saw 22\nSkipping line 158070: expected 15 fields, saw 22\nSkipping line 166007: expected 15 fields, saw 22\nSkipping line 171877: expected 15 fields, saw 22\nSkipping line 177756: expected 15 fields, saw 22\nSkipping line 181773: expected 15 fields, saw 22\nSkipping line 191085: expected 15 fields, saw 22\nSkipping line 196273: expected 15 fields, saw 22\nSkipping line 196331: expected 15 fields, saw 22\n'
b'Skipping line 197000: expected 15 fields, saw 22\nSkipping line 197011: expected 15 fields, saw 22\nSkipping line 197432: expected 15 fields, saw 22\nSkipping line 208016: expected 15 fields, saw 22\nSkipping line 214110: expected 15 fields, saw 22\nSkipping line 244328: expected 15 fields, saw 22\nSkipping line 248519: expected 15 fields,

In [3]:
df.head()

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
0,US,3653882,R3O9SGZBVQBV76,B00FALQ1ZC,937001370,"Invicta Women's 15150 ""Angel"" 18k Yellow Gold ...",Watches,5,0,0,N,Y,Five Stars,Absolutely love this watch! Get compliments al...,2015-08-31
1,US,14661224,RKH8BNC3L5DLF,B00D3RGO20,484010722,Kenneth Cole New York Women's KC4944 Automatic...,Watches,5,0,0,N,Y,I love thiswatch it keeps time wonderfully,I love this watch it keeps time wonderfully.,2015-08-31
2,US,27324930,R2HLE8WKZSU3NL,B00DKYC7TK,361166390,Ritche 22mm Black Stainless Steel Bracelet Wat...,Watches,2,1,1,N,Y,Two Stars,Scratches,2015-08-31
3,US,7211452,R31U3UH5AZ42LL,B000EQS1JW,958035625,Citizen Men's BM8180-03E Eco-Drive Stainless S...,Watches,5,0,0,N,Y,Five Stars,"It works well on me. However, I found cheaper ...",2015-08-31
4,US,12733322,R2SV659OUJ945Y,B00A6GFD7S,765328221,Orient ER27009B Men's Symphony Automatic Stain...,Watches,4,0,0,N,Y,"Beautiful face, but cheap sounding links",Beautiful watch face. The band looks nice all...,2015-08-31


In [4]:
# Remove missing value
df.dropna(subset=['review_body'],inplace=True)

In [5]:
df.reset_index(inplace=True, drop=True)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 960056 entries, 0 to 960055
Data columns (total 15 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   marketplace        960056 non-null  object
 1   customer_id        960056 non-null  int64 
 2   review_id          960056 non-null  object
 3   product_id         960056 non-null  object
 4   product_parent     960056 non-null  int64 
 5   product_title      960054 non-null  object
 6   product_category   960056 non-null  object
 7   star_rating        960056 non-null  int64 
 8   helpful_votes      960056 non-null  int64 
 9   total_votes        960056 non-null  int64 
 10  vine               960056 non-null  object
 11  verified_purchase  960056 non-null  object
 12  review_headline    960049 non-null  object
 13  review_body        960056 non-null  object
 14  review_date        960052 non-null  object
dtypes: int64(5), object(10)
memory usage: 109.9+ MB


In [7]:
# use the first 1000 data as our training data
data = df.loc[:999, 'review_body'].tolist()

In [8]:
data[:10]

['Absolutely love this watch! Get compliments almost every time I wear it. Dainty.',
 'I love this watch it keeps time wonderfully.',
 'Scratches',
 'It works well on me. However, I found cheaper prices in other places after making the purchase',
 "Beautiful watch face.  The band looks nice all around.  The links do make that squeaky cheapo noise when you swing it back and forth on your wrist which can be embarrassing in front of watch enthusiasts.  However, to the naked eye from afar, you can't tell the links are cheap or folded because it is well polished and brushed and the folds are pretty tight for the most part.<br /><br />I love the new member of my collection and it looks great.  I've had it for about a week and so far it has kept good time despite day 1 which is typical of a new mechanical watch",
 'i love this watch for my purpose, about the people complaining should of done their research better before buying. dumb people.',
 'for my wife and she loved it, looks great and a 

# Part 2: Tokenizing and Stemming

Load stopwords and stemmer function from NLTK library.
Stop words are words like "a", "the", or "in" which don't convey significant meaning.
Stemming is the process of breaking a word down into its root.

In [9]:
# Use nltk's English stopwords.
stopwords = nltk.corpus.stopwords.words('english') #stopwords.append("n't")
stopwords.append("'s")
stopwords.append("'m")
stopwords.append("br") #html <br>
stopwords.append("watch")

print ("We use " + str(len(stopwords)) + " stop-words from nltk library.")
print (stopwords[:10])

We use 183 stop-words from nltk library.
['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]


Use our defined functions to analyze (i.e. tokenize, stem) our reviews.

In [10]:
from nltk.stem.snowball import SnowballStemmer
# from nltk.stem import WordNetLemmatizer 

stemmer = SnowballStemmer("english")

# tokenization and stemming
def tokenization_and_stemming(text):
    tokens = []
    # exclude stop words and tokenize the document, generate a list of string 
    for word in nltk.word_tokenize(text):
        if word.lower() not in stopwords:
            tokens.append(word.lower())

    filtered_tokens = []
    
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if token.isalpha():
            filtered_tokens.append(token)
            
    # stemming
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems

In [11]:
tokenization_and_stemming(data[0])

['absolut',
 'love',
 'get',
 'compliment',
 'almost',
 'everi',
 'time',
 'wear',
 'dainti']

In [12]:
data[0]

'Absolutely love this watch! Get compliments almost every time I wear it. Dainty.'

# Part 3: TF-IDF

TF: Term Frequency

IDF: Inverse Document Frequency

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer
# define vectorizer parameters
# TfidfVectorizer will help us to create tf-idf matrix
# max_df : maximum document frequency for the given word
# min_df : minimum document frequency for the given word
# max_features: maximum number of words
# use_idf: if not true, we only calculate tf
# stop_words : built-in stop words
# tokenizer: how to tokenize the document
# ngram_range: (min_value, max_value), eg. (1, 3) means the result will include 1-gram, 2-gram, 3-gram
tfidf_model = TfidfVectorizer(max_df=0.99, max_features=1000,
                                 min_df=0.01, stop_words='english',
                                 use_idf=True, tokenizer=tokenization_and_stemming, ngram_range=(1,1))

tfidf_matrix = tfidf_model.fit_transform(data) #fit the vectorizer to synopses

print ("In total, there are " + str(tfidf_matrix.shape[0]) + \
      " reviews and " + str(tfidf_matrix.shape[1]) + " terms.")

C:\Users\86186\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
C:\Users\86186\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['afterward', 'alon', 'alreadi', 'alway', 'anoth', 'anyon', 'anyth', 'anywher', 'becam', 'becom', 'besid', 'cri', 'describ', 'els', 'elsewher', 'empti', 'everi', 'everyon', 'everyth', 'everywher', 'fifti', 'forti', 'henc', 'hereaft', 'herebi', 'howev', 'hundr', 'inde', 'mani', 'meanwhil', 'moreov', 'nobodi', 'noon', 'noth', 'nowher', 'otherwis', 'perhap', 'pleas', 'sever', 'sinc', 'sincer', 'sixti', 'someon', 'someth', 'sometim', 'somewher', 'thenc', 'thereaft', 'therebi', 'therefor', 'togeth', 'twelv', 'twenti', 'whatev', 'whenc', 'whenev', 'wherea', 'whereaft', 'wherebi', 'wherev'] not in stop_words.

In total, there are 1000 reviews and 239 terms.


In [14]:
tfidf_matrix

<1000x239 sparse matrix of type '<class 'numpy.float64'>'
	with 6891 stored elements in Compressed Sparse Row format>

In [15]:
tfidf_matrix.toarray() #todense()

array([[0.       , 0.5125863, 0.       , ..., 0.       , 0.       ,
        0.       ],
       [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ],
       [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ],
       ...,
       [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ],
       [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ],
       [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ]])

In [16]:
tfidf_matrix.todense()

matrix([[0.       , 0.5125863, 0.       , ..., 0.       , 0.       ,
         0.       ],
        [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
         0.       ],
        [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
         0.       ],
        ...,
        [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
         0.       ],
        [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
         0.       ],
        [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
         0.       ]])

In [17]:
print(type(tfidf_matrix.toarray()))

<class 'numpy.ndarray'>


In [18]:
print(type(tfidf_matrix.todense()))

<class 'numpy.matrix'>


Save the terms identified by TF-IDF.

In [19]:
# words
tf_selected_words = tfidf_model.get_feature_names_out()

In [20]:
# print out words
tf_selected_words

array(['abl', 'absolut', 'accur', 'actual', 'adjust', 'alarm', 'alreadi',
       'alway', 'amaz', 'amazon', 'anoth', 'arm', 'arriv', 'automat',
       'awesom', 'bad', 'band', 'batteri', 'beauti', 'best', 'better',
       'big', 'bit', 'black', 'blue', 'bought', 'box', 'bracelet',
       'brand', 'break', 'bright', 'broke', 'button', 'buy', 'ca', 'came',
       'case', 'casio', 'chang', 'cheap', 'clasp', 'classi', 'clock',
       'color', 'come', 'comfort', 'compliment', 'cool', 'cost', 'crown',
       'crystal', 'dark', 'date', 'daughter', 'day', 'deal', 'definit',
       'deliveri', 'design', 'dial', 'differ', 'difficult', 'disappoint',
       'display', 'dress', 'durabl', 'easi', 'easili', 'end', 'everi',
       'everyday', 'everyth', 'exact', 'excel', 'expect', 'expens',
       'face', 'fair', 'far', 'fast', 'featur', 'feel', 'fell', 'fine',
       'finish', 'fit', 'function', 'gave', 'gift', 'gold', 'good', 'got',
       'great', 'hand', 'happi', 'hard', 'heavi', 'high', 'hold',
 

# Part 4: K-means clustering

In [21]:
# k-means clustering
from sklearn.cluster import KMeans

num_clusters = 5

# number of clusters
km = KMeans(n_clusters=num_clusters)
km.fit(tfidf_matrix)

clusters = km.labels_.tolist()

C:\Users\86186\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


## 4.1. Analyze K-means Result

In [22]:
# create DataFrame films from all of the input files.
product = { 'review': df[:1000].review_body, 'cluster': clusters}
frame = pd.DataFrame(product, columns = ['review', 'cluster'])

In [23]:
frame.head(10)

,review,cluster
0,Absolutely love this watch! Get compliments al...,3
1,I love this watch it keeps time wonderfully.,3
2,Scratches,4
3,"It works well on me. However, I found cheaper ...",4
4,Beautiful watch face. The band looks nice all...,4
5,"i love this watch for my purpose, about the pe...",3
6,"for my wife and she loved it, looks great and ...",0
7,I was about to buy this thinking it was a Swis...,4
8,Watch is perfect. Rugged with the metal &#34;B...,0
9,Great quality and build.<br />The motors are r...,4


In [24]:
print ("Number of reviews included in each cluster:")
frame['cluster'].value_counts().to_frame()

Number of reviews included in each cluster:


,cluster
4,667
3,111
0,85
2,75
1,62


In [25]:
km.cluster_centers_

array([[0.00379401, 0.        , 0.        , ..., 0.00262203, 0.        ,
        0.02286126],
       [0.        , 0.        , 0.        , ..., 0.        , 0.00784247,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.00874405,
        0.        ],
       [0.        , 0.04125958, 0.        , ..., 0.01222159, 0.01717547,
        0.00400254],
       [0.00565643, 0.00437174, 0.00382872, ..., 0.00653108, 0.01801031,
        0.01380752]])

In [26]:
km.cluster_centers_.shape

(5, 239)

In [38]:
print ("<Document clustering result by K-means>")

#km.cluster_centers_ denotes the importances of each items in centroid.
#We need to sort it in decreasing-order and get the top k items.
order_centroids = km.cluster_centers_.argsort()[:, ::-1] 

Cluster_keywords_summary = {}
for i in range(num_clusters):
    print ("Cluster " + str(i) + " words:", end='')
    Cluster_keywords_summary[i] = []
    for ind in order_centroids[i, :6]: 
        Cluster_keywords_summary[i].append(tf_selected_words[ind])
        print (tf_selected_words[ind] + ",", end='')
#     print ()
    
    cluster_reviews = frame[frame.cluster==i].review.tolist()
#     print ("Cluster " + str(i) + " reviews (" + str(len(cluster_reviews)) + " reviews): ")
#     print (", ".join(cluster_reviews))
#     print ()

<Document clustering result by K-means>
Cluster 0 words:great,look,price,work,product,comfort,Cluster 1 words:nice,price,look,realli,simpl,good,Cluster 2 words:good,product,price,qualiti,love,work,Cluster 3 words:love,wife,look,husband,absolut,beauti,Cluster 4 words:look,like,work,band,time,beauti,

# Part 5: Topic Modeling - Latent Dirichlet Allocation

In [39]:
# Use LDA for clustering
from sklearn.decomposition import LatentDirichletAllocation
lda = LatentDirichletAllocation(n_components=5)

In [40]:
# document topic matrix for tfidf_matrix_lda
lda_output = lda.fit_transform(tfidf_matrix)
print(lda_output.shape)
print(lda_output)

(1000, 5)
[[0.20973007 0.06251476 0.23797785 0.05996152 0.4298158 ]
 [0.08421691 0.08887731 0.08585954 0.08378519 0.65726106]
 [0.2        0.2        0.2        0.2        0.2       ]
 ...
 [0.10000029 0.10010384 0.59989532 0.10000031 0.10000024]
 [0.06688115 0.06848949 0.07199443 0.71615793 0.07647699]
 [0.06881643 0.72740367 0.06737028 0.06723656 0.06917306]]


In [41]:
# topics and words matrix
topic_word = lda.components_
print(topic_word.shape)
print(topic_word)

(5, 239)
[[ 0.20100798  0.20085356  0.20036621 ...  0.20292827  1.07014988
   2.35316522]
 [ 2.59298323  0.21035856  2.75185389 ...  1.98582976 11.03571106
   9.63319722]
 [ 0.20012333  0.2005438   0.20012406 ...  0.20108446  3.16770653
   0.20914669]
 [ 1.90038756  0.25580347  0.20137576 ...  4.34508237  0.58592423
   0.20134701]
 [ 0.20082497  7.62820344  0.20003517 ...  0.20077245  0.2018961
   0.20024932]]


In [42]:
# column names
topic_names = ["Topic" + str(i) for i in range(lda.n_components)]

# index names
doc_names = ["Doc" + str(i) for i in range(len(data))]

df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topic_names, index=doc_names)

# get dominant topic for each document
topic = np.argmax(df_document_topic.values, axis=1)
df_document_topic['topic'] = topic

df_document_topic.head(10)

,Topic0,Topic1,Topic2,Topic3,Topic4,topic
Doc0,0.21,0.06,0.24,0.06,0.43,4
Doc1,0.08,0.09,0.09,0.08,0.66,4
Doc2,0.20,0.20,0.20,0.20,0.20,0
Doc3,0.06,0.76,0.06,0.06,0.06,1
Doc4,0.04,0.84,0.04,0.04,0.04,1
Doc5,0.07,0.08,0.68,0.08,0.10,2
Doc6,0.70,0.06,0.06,0.06,0.11,0
Doc7,0.06,0.75,0.06,0.06,0.06,1
Doc8,0.21,0.66,0.04,0.04,0.04,1
Doc9,0.06,0.77,0.06,0.06,0.06,1


In [43]:
df_document_topic['topic'].value_counts().to_frame()

,topic
1,417
0,165
4,157
2,149
3,112


In [44]:
# topic word matrix
print(lda.components_)
# topic-word matrix
df_topic_words = pd.DataFrame(lda.components_)

# column and index
df_topic_words.columns = tfidf_model.get_feature_names_out()
df_topic_words.index = topic_names

df_topic_words.head()

[[ 0.20100798  0.20085356  0.20036621 ...  0.20292827  1.07014988
   2.35316522]
 [ 2.59298323  0.21035856  2.75185389 ...  1.98582976 11.03571106
   9.63319722]
 [ 0.20012333  0.2005438   0.20012406 ...  0.20108446  3.16770653
   0.20914669]
 [ 1.90038756  0.25580347  0.20137576 ...  4.34508237  0.58592423
   0.20134701]
 [ 0.20082497  7.62820344  0.20003517 ...  0.20077245  0.2018961
   0.20024932]]


,abl,absolut,accur,actual,adjust,alarm,alreadi,alway,amaz,amazon,...,weight,went,wife,wind,wish,work,worn,worth,wrist,year
Topic0,0.201008,0.200854,0.200366,1.351726,0.201727,0.200107,0.200460,2.029179,0.204513,0.212412,...,0.204188,0.200985,11.836941,0.200073,0.254360,12.669303,0.200454,0.202928,1.070150,2.353165
Topic1,2.592983,0.210359,2.751854,3.130763,5.488979,3.451471,0.211176,2.182212,0.207808,5.604948,...,1.845533,2.915399,0.202791,3.430855,3.058911,18.779266,2.793635,1.985830,11.035711,9.633197
Topic2,0.200123,0.200544,0.200124,0.201253,0.200225,1.886171,0.203324,0.201593,0.202471,0.201022,...,3.266989,0.200271,0.200228,0.200079,0.202336,5.794641,0.200380,0.201084,3.167707,0.209147
Topic3,1.900388,0.255803,0.201376,0.205139,0.201141,0.202597,4.947876,0.225816,0.989908,0.583345,...,0.200353,0.200462,0.200884,0.202000,0.854161,3.617281,0.200116,4.345082,0.585924,0.201347
Topic4,0.200825,7.628203,0.200035,0.967516,0.201099,0.200066,0.200533,0.201204,9.164135,0.200813,...,0.201299,0.200057,0.201554,0.200701,0.200741,0.201176,0.808508,0.200772,0.201896,0.200249


In [45]:
# print top n keywords for each topic
def print_topic_words(tfidf_model, lda_model, n_words):
    words = np.array(tfidf_model.get_feature_names_out())
    topic_words = []
    # for each topic, we have words weight
    for topic_words_weights in lda_model.components_:
        top_words = topic_words_weights.argsort()[::-1][:n_words]
        topic_words.append(words.take(top_words))
    return topic_words

topic_keywords = print_topic_words(tfidf_model=tfidf_model, lda_model=lda, n_words=15)        

df_topic_words = pd.DataFrame(topic_keywords)
df_topic_words.columns = ['Word '+str(i) for i in range(df_topic_words.shape[1])]
df_topic_words.index = ['Topic '+str(i) for i in range(df_topic_words.shape[0])]
df_topic_words

,Word 0,Word 1,Word 2,Word 3,Word 4,Word 5,Word 6,Word 7,Word 8,Word 9,Word 10,Word 11,Word 12,Word 13,Word 14
Topic 0,great,excel,perfect,work,look,wife,price,deal,thank,want,ship,compliment,cool,feel,fit
Topic 1,time,look,band,work,like,wear,use,face,day,easi,great,big,size,read,hand
Topic 2,good,like,band,strap,look,fine,qualiti,price,buy,invicta,leather,work,watch,time,long
Topic 3,product,expect,beauti,seller,deliveri,stylish,exact,clock,price,batteri,fast,box,better,pleas,pictur
Topic 4,love,nice,realli,awesom,amaz,gift,beauti,absolut,pretti,bought,recommend,super,light,look,wear
